In [1]:
# import required libraries
import json
import csv
import pandas as pd
from rdflib import Graph

In [2]:
# Define a function to extract some information of transaction generated by DEPCHA Bookkeeping Ontology into CSV file
# The information to be extracted are the following: Transaction ID, Business Partner, Type of service, and Amount of payment in pence

def summarize_transaction_from_DEPCHA_rdf(parse_file, output_file):

    g = Graph()
    g.parse(parse_file)

    qres = g.query("""
        PREFIX bk: <https://gams.uni-graz.at/o:depcha.bookkeeping#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    
        SELECT distinct ?transaction ?transfer ?from ?to ?commodity ?quantity ?unit
        WHERE {
            ?transaction rdf:type bk:Transaction;
                         bk:consistsOf ?transfer.

            ?transfer bk:from ?from;
                      bk:to ?to;
                      bk:transfers ?measure.

        OPTIONAL{?measure bk:commodity ?commodity}.
        OPTIONAL{?measure bk:quantity ?quantity}.
        OPTIONAL{?measure bk:unit ?unit}.
        }""")

    transactionID_check = 'test'
    business_partner = ''
    service = ''
    total_pence = 0
    tab = '\t'

    f_output = open(output_file, 'w', encoding='utf-8')
    f_output.write('TransactionID\tBusiness_partner\tService\tPayment\n')

    for row in qres:
        transactionID = row[0]
    
        # When Transaction ID is changed
        if transactionID != transactionID_check:
            if transactionID_check != "test":
                f_output.write(f'{transactionID_check}{tab}{business_partner}{tab}{service}{tab}{total_pence}')
                f_output.write('\n')
            total_pence = 0
            business_partner = ''
            service = ''

            if str(row[-3]) != "Currency":
                business_partner = row[2]
                service = row[-3]
            elif str(row[-3]) == "Currency":
                if str(row[-1]) == "pound":
                    total_pence += int(row[-2]) * 240
                elif str(row[-1]) == "shilling":
                    total_pence += int(row[-2]) * 12
                elif str(row[-1]) == "pence":
                    total_pence += int(row[-2])
    
        # When Transaction ID is not changed
        elif transactionID == transactionID_check:
            if str(row[-3]) != "Currency":
                business_partner = row[2]
                service = row[-3]
            elif str(row[-3]) == "Currency":
                if str(row[-1]) == "pound":
                    total_pence += int(row[-2]) * 240
                elif str(row[-1]) == "shilling":
                    total_pence += int(row[-2]) * 12
                elif str(row[-1]) == "pence":
                    total_pence += int(row[-2])
            
        transactionID_check = row[0]
    
    f_output.write(f'{transactionID_check}{tab}{business_partner}{tab}{service}{tab}{total_pence}')
    f_output.write('\n')    
    f_output.close()


In [3]:
# Define a function to calculate the sum of payment according to the names of company and service

def calculate_sum_of_payment_by_firm_and_service(data_frame, company, service):
    result = sum(data_frame[(data_frame['Business_partner'] == company) & (data_frame['Service'] == service)].Payment)
    return result

In [4]:
# Define a function to generate a dictionary object storing the names of company and service

def zip_firm_and_service_to_dict(data_frame):
    firms = list(data_frame['Business_partner'])
    services = list(data_frame['Service'])
    dictionary = dict(zip(firms, services))

    return dictionary


In [5]:
# Function to adjust the format of CSV file for the later process

def adjust_csv_for_generating_json(firm_service_dict, data_frame, output_file):
    tab = "\t"

    f_output = open(output_file, 'w', encoding='utf-8')

    for company, service in firm_service_dict.items():
        value = calculate_sum_of_payment_by_firm_and_service(data_frame, company, service)
        f_output.write(f'Types of Service{tab}{service}{tab}{company}{tab}{value}')
        f_output.write('\n')
    
    f_output.close()


In [6]:
# Function to create a JSON file for visualising the treemap

def generate_json_from_csv(parse_file, output_file):

    f = open(parse_file, 'r', encoding='utf-8')
    reader = csv.reader(f, delimiter='\t')

    top = {}
    service_dict = {}

    for row in reader:
        head, service, company, value = row[0], row[1], row[2], row[3]
        top.setdefault(head, [])
        if service not in top[head]:
            top[head].append(service)
        if service not in service_dict.keys():
            service_dict[service] = []
        service_dict[service].append({"name": company, "value": int(value)})

    result_list = []
    base = {}

    for item in top[head]:
        base['name'] = item
        base['children'] = service_dict[item]
        result_list.append(base)
        base = {}
    
    result = {'name': head, 'children': result_list}
    final = [result]

    with open(output_file, 'w', encoding='utf-8') as rslt:
        json.dump(final, rslt)


In [12]:
# Function to create an HTML file for visualising a treemap developed by anychart.js
# https://docs.anychart.com/Quick_Start/Quick_Start

def create_html_for_treemap(json_file, output_html):
    input_file = open(json_file, 'r', encoding='utf-8')
    whole_text = input_file.read()
    input_file.close()
    
    html_part1 = """
<html>
<head>
  <script src="https://cdn.anychart.com/releases/8.6.0/js/anychart-base.min.js" type="text/javascript"></script>
  <script src="https://cdn.anychart.com/releases/8.6.0/js/anychart-core.min.js"></script>
  <script src="https://cdn.anychart.com/releases/8.6.0/js/anychart-treemap.min.js"></script>
</head>
<body>
  <div id="container" style="width: 1500px; height: 700px;"></div>
  <script>
anychart.onDocumentReady(function () {
    // create data
    var data = """
    
    html_part2 = """
// create a chart and set the data
    chart = anychart.treeMap(data, "as-tree");

// set the maximum depth of levels shown
    chart.maxDepth(3);

// configure the text of headers in the hovered state
    chart.hovered().headers().format("{%value}");

// configure the font of headers
    chart.normal().headers().fontColor("#990000");
    chart.normal().headers().fontSize("14");
    chart.normal().headers().fontWeight('bold');
    chart.hovered().headers().fontColor("#000099");

// set the chart title
    chart.title().useHtml(true);
    chart.title("Treemap: Business Partners of T.W.Ward<br><br>" +
                "<span style='font-size:12; font-style:italic'>" +
                "Visualised with anychart.js</span>");

// set the container id
    chart.container("container");

// initiate drawing the chart
    chart.draw();
});
    
// enable/disable headers
function headersEnabled(enabled) {
  chart.headers(enabled);}
  </script>
</body>"""
    
    output_file = open(output_html, 'w', encoding='utf-8')
    enter = '\n'
    output_file.write(f'{html_part1}{whole_text};{enter}{html_part2}')
    output_file.close()


In [13]:
# Execute!

summarize_transaction_from_DEPCHA_rdf('0810nakamura_consult.xml', 'output_for_treemap.tsv')
data_frame = pd.read_csv('output_for_treemap.tsv', sep='\t', header=0)

firm_service_dict = zip_firm_and_service_to_dict(data_frame)
adjust_csv_for_generating_json(firm_service_dict, data_frame, 'DEPCHA_tsv_for_json.tsv')
generate_json_from_csv('DEPCHA_tsv_for_json.tsv', '0813treemap.json')
create_html_for_treemap('0813treemap.json', 'treemap.html')
